In [18]:
import requests
import pandas as pd

def extrair_dados(salvar_arquivos=True):
    url = "https://restcountries.com/v3.1/all"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        # Lista principal
        dados_paises = []

        # Listas secundárias para idiomas e moedas
        dados_idiomas = []
        dados_moedas = []

        for pais in data:
            nome = pais.get('name', {}).get('common')
            capital = pais.get('capital', [None])[0] if pais.get('capital') else None
            regiao = pais.get('region')
            subregiao = pais.get('subregion')
            area = pais.get('area')
            populacao = pais.get('population')
            bandeira = pais.get('flags', {}).get('png')

            # idiomas em string para visualização geral
            idiomas_dict = pais.get('languages', {})
            
            # Moedas: siglas separadas por vírgula
            moedas_dict = pais.get('currencies', {})
            
            # Adiciona à base de países
            dados_paises.append({
                'Nome': nome,
                'Capital': capital,
                'Região': regiao,
                'Sub-Região': subregiao,
                'Área (km²)': area,
                'População': populacao,
                'Bandeira': bandeira
            })

            # Cria uma linha por idioma
            for idioma in idiomas_dict.values():
                dados_idiomas.append({
                    'País': nome,
                    'Idioma': idioma
                })

            # Cria uma linha por moeda
            for sigla, moeda_info in moedas_dict.items():
                nome_moeda = moeda_info.get('name', '')
                simbolo = moeda_info.get('symbol', '')
                dados_moedas.append({
                    'País': nome,
                    'Sigla': sigla,
                    'Moeda': nome_moeda,
                    'Símbolo': simbolo
                })

        # Criação dos DataFrames
        df_paises = pd.DataFrame(dados_paises)
        df_idiomas = pd.DataFrame(dados_idiomas)
        df_moedas = pd.DataFrame(dados_moedas)

        # Salva os arquivos
        if salvar_arquivos:
            df_paises.to_csv('Base_Dados.csv', index=False)
            df_idiomas.to_csv('Base_Dados_Idiomas.csv', index=False)
            df_moedas.to_csv('Base_Dados_Moedas.csv', index=False)
            print("✅ Arquivos 'paises.csv' e 'paises_idiomas.csv' salvos com sucesso!")

        return df_paises, df_idiomas

    except requests.exceptions.RequestException as e:
        print("❌ Erro ao acessar a API:", e)
        return None, None
